# Data Cleaning

**Now that we've obtained data from the website, it's important to note that it's not yet in a clean and analysis-ready state. The reviews section requires preprocessing to handle punctuation, spelling, and other characters.**

CSV File Utilized: BA_Reviews.csv

<hr>

## Imports

`pandas` for Data Manipulation <br>
`os` for System Interaction <br>
`re` for Regular Expressions

In [1]:
import pandas as pd
import os
import re

## Creating a DataFrame from `csv` file

In [2]:
cwd = os.getcwd()

df = pd.read_csv(cwd + "/BA_Reviews.csv", index_col = 0)

In [3]:
df.head(5)

,Review,Rating,Date,Country
0,Not Verified | Flew back from Malta after sc...,5.0,22nd October 2023,United Kingdom
1,Not Verified | Cabin luggage had to go to carg...,8.0,21st October 2023,Netherlands
2,✅ Trip Verified | I have been using BA for a ...,3.0,21st October 2023,United Kingdom
3,✅ Trip Verified | I flew from Istanbul to Lo...,4.0,19th October 2023,United Kingdom
4,Not Verified | I have flow on BA several time...,1.0,19th October 2023,United States


<hr>


## Review Column

**Checking which users are `Verified`**

In [4]:
df['Verified'] = df.Review.str.contains('Trip Verified')

df.head(5)

,Review,Rating,Date,Country,Verified
0,Not Verified | Flew back from Malta after sc...,5.0,22nd October 2023,United Kingdom,False
1,Not Verified | Cabin luggage had to go to carg...,8.0,21st October 2023,Netherlands,False
2,✅ Trip Verified | I have been using BA for a ...,3.0,21st October 2023,United Kingdom,True
3,✅ Trip Verified | I flew from Istanbul to Lo...,4.0,19th October 2023,United Kingdom,True
4,Not Verified | I have flow on BA several time...,1.0,19th October 2023,United States,False


**Cleaning Reviews:**
**We will extract the column of reviews into a separate dataframe and clean it for semantic analysis.**

In [5]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('wordnet')

# This lemmatizer is used to reduce words to their base or root form.
lemma = WordNetLemmatizer()

# Extracting Review Text from Review Column
review_data = []

for review_text in df['Review']:
    if '|' in review_text:
        parts = review_text.split('|')
        review_data.append(parts[1])
    else:
        review_data.append(review_text)
        

# Loop through each review, remove punctuations, convert to lowercase, split, lemmatize, and add to corpus
corpus = []
for rev in review_data:
    rev = re.sub('[^a-zA-Z]', ' ', rev)
    rev = rev.lower()
    rev = rev.split()
    rev = [lemma.lemmatize(word) for word in rev if word not in set(stopwords.words("english"))]
    rev = " ".join(rev)
    corpus.append(rev)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/pragatimehra/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/pragatimehra/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:
df['Corpus'] = corpus

    Set Pandas display options to show all rows and columns
        pd.set_option('display.max_rows', None)
        pd.set_option('display.max_columns', None)
        
    Reset using
        pd.reset_option('display.max_rows', None)
        pd.reset_option('display.max_columns', None)

In [7]:
df.head(5)

,Review,Rating,Date,Country,Verified,Corpus
0,Not Verified | Flew back from Malta after sc...,5.0,22nd October 2023,United Kingdom,False,flew back malta scattering son ash favourite s...
1,Not Verified | Cabin luggage had to go to carg...,8.0,21st October 2023,Netherlands,False,cabin luggage go cargo even said carried medic...
2,✅ Trip Verified | I have been using BA for a ...,3.0,21st October 2023,United Kingdom,True,using ba average cannot say pretty satisfied b...
3,✅ Trip Verified | I flew from Istanbul to Lo...,4.0,19th October 2023,United Kingdom,True,flew istanbul london business class half fligh...
4,Not Verified | I have flow on BA several time...,1.0,19th October 2023,United States,False,flow ba several time since airline started non...


<hr>

## Date Column

In [8]:
df.Date.dtypes

dtype('O')

**Convert the date to datetime format**

In [9]:
# Function to parse date strings and format them as dd-mm-yyyy
def format_date(date_str):
    parts = date_str.split()
    day = int(parts[0].rstrip('stndrdth'))  # Remove ordinal indicators
    month = pd.to_datetime(parts[1], format='%B').month
    year = int(parts[2])
    return f"{day:02d}-{month:02d}-{year}"

df['Formatted Date'] = df['Date'].apply(format_date)

In [10]:
df.head(5)

,Review,Rating,Date,Country,Verified,Corpus,Formatted Date
0,Not Verified | Flew back from Malta after sc...,5.0,22nd October 2023,United Kingdom,False,flew back malta scattering son ash favourite s...,22-10-2023
1,Not Verified | Cabin luggage had to go to carg...,8.0,21st October 2023,Netherlands,False,cabin luggage go cargo even said carried medic...,21-10-2023
2,✅ Trip Verified | I have been using BA for a ...,3.0,21st October 2023,United Kingdom,True,using ba average cannot say pretty satisfied b...,21-10-2023
3,✅ Trip Verified | I flew from Istanbul to Lo...,4.0,19th October 2023,United Kingdom,True,flew istanbul london business class half fligh...,19-10-2023
4,Not Verified | I have flow on BA several time...,1.0,19th October 2023,United States,False,flow ba several time since airline started non...,19-10-2023


<hr>

## Rating Column

In [11]:
df.Rating.unique()

array([ 5.,  8.,  3.,  4.,  1.,  2., 10.,  6.,  7.,  9., nan])

**Checking NaN Rows**

In [12]:
df[pd.isna(df['Rating'])]

,Review,Rating,Date,Country,Verified,Corpus,Formatted Date
3144,Bangkok to London Heathrow. 8 Sep 2014. On thi...,NaN,23rd January 2015,United Kingdom,False,bangkok london heathrow sep flight british air...,23-01-2015
3272,Flew with my wife LHR-JNB BA055 on an A380 Oct...,NaN,20th November 2014,France,False,flew wife lhr jnb ba october st back jnb lhr b...,20-11-2014
3294,31.10.14 - LHR to Berlin Tegel. Flight out goo...,NaN,6th November 2014,United Kingdom,False,lhr berlin tegel flight good modern plane clea...,06-11-2014
3527,Just taken a BA flight LHR-JFK rtn. Outstandin...,NaN,13th July 2014,United Kingdom,False,taken ba flight lhr jfk rtn outstanding servic...,13-07-2014
3559,Flight BA 202 Boston to Heathrow. Aircraft B74...,NaN,23rd June 2014,United Kingdom,False,flight ba boston heathrow aircraft b version s...,23-06-2014


**Removing NaN valued Rows**

In [13]:
df.drop(df[pd.isna(df['Rating'])].index, inplace=True)

In [14]:
df[pd.isna(df['Rating'])]

,Review,Rating,Date,Country,Verified,Corpus,Formatted Date


<hr>

## Country Column

In [15]:
df.Country.unique()

array(['United Kingdom', 'Netherlands', 'United States', 'Australia',
       'Italy', 'France', 'Germany', 'Switzerland', 'South Africa',
       'Canada', 'Kuwait', 'Iceland', 'Denmark', 'Poland', 'Hong Kong',
       'Qatar', 'Spain', 'Greece', 'Senegal', 'United Arab Emirates',
       'Romania', 'Cyprus', 'Chile', 'Sweden', 'Ireland', 'Austria',
       'India', 'Belgium', 'New Zealand', 'Czech Republic', 'Malaysia',
       'Singapore', 'Ghana', 'Bermuda', 'Botswana', 'Brazil', 'Panama',
       'Nigeria', 'Russian Federation', 'Philippines', 'Bulgaria',
       'Thailand', 'Argentina', 'Mexico', 'Saint Kitts and Nevis',
       'Vietnam', 'Norway', 'Jordan', 'Japan', 'Taiwan', 'China',
       'Slovakia', 'Israel', 'South Korea', 'Saudi Arabia', 'Hungary',
       'Portugal', 'Cayman Islands', 'Costa Rica', 'Egypt', 'Laos',
       'Turkey', 'Indonesia', 'Bahrain', 'Dominican Republic',
       'Luxembourg', 'Finland', 'Ukraine', nan, 'Trinidad & Tobago',
       'Barbados', 'Oman'], dtype=ob

**Checking NaN Rows**

In [16]:
df[pd.isna(df['Country'])]

,Review,Rating,Date,Country,Verified,Corpus,Formatted Date
3041,I travelled from London to Jo'burg and back on...,1.0,8th April 2015,NaN,False,travelled london jo burg back airbus mixed exp...,08-04-2015
3347,St Lucia to London round trip. Full flight bot...,9.0,20th October 2014,NaN,False,st lucia london round trip full flight way qui...,20-10-2014


In [17]:
df.drop(df[pd.isna(df['Country'])].index, inplace=True)

In [18]:
df[pd.isna(df['Rating'])]

,Review,Rating,Date,Country,Verified,Corpus,Formatted Date


<hr>

## Resetting the Index

In [19]:
df.reset_index(drop=True)

,Review,Rating,Date,Country,Verified,Corpus,Formatted Date
0,Not Verified | Flew back from Malta after sc...,5.0,22nd October 2023,United Kingdom,False,flew back malta scattering son ash favourite s...,22-10-2023
1,Not Verified | Cabin luggage had to go to carg...,8.0,21st October 2023,Netherlands,False,cabin luggage go cargo even said carried medic...,21-10-2023
2,✅ Trip Verified | I have been using BA for a ...,3.0,21st October 2023,United Kingdom,True,using ba average cannot say pretty satisfied b...,21-10-2023
3,✅ Trip Verified | I flew from Istanbul to Lo...,4.0,19th October 2023,United Kingdom,True,flew istanbul london business class half fligh...,19-10-2023
4,Not Verified | I have flow on BA several time...,1.0,19th October 2023,United States,False,flow ba several time since airline started non...,19-10-2023
...,...,...,...,...,...,...,...
3673,This was a bmi Regional operated flight on a R...,8.0,29th August 2012,United Kingdom,False,bmi regional operated flight rj manchester hea...,29-08-2012
3674,LHR to HAM. Purser addresses all club passenge...,2.0,28th August 2012,United Kingdom,False,lhr ham purser address club passenger name boa...,28-08-2012
3675,My son who had worked for British Airways urge...,7.0,12th October 2011,United Kingdom,False,son worked british airway urged fly british ai...,12-10-2011
3676,London City-New York JFK via Shannon on A318 b...,1.0,11th October 2011,United States,False,london city new york jfk via shannon really ni...,11-10-2011


<hr>

## Export the Cleaned CSV

In [20]:
df.to_csv(cwd + "/Cleaned-BA-Reviews.csv")